In [1]:
!pip install pydub


In [14]:
soundfiles = {
    "2" : {
        "C" : 'C2_tone.wav',
        "C#": 'C#2_tone.wav',
        "D" : 'D2_tone.wav',
        "D#": 'D#2_tone.wav',
        "E" : 'E2_tone.wav',
        "F" : 'F2_tone.wav',
        "F#": 'F#2_tone.wav',
        "G" : 'G2_tone.wav',
        "G#": 'G#2_tone.wav',
        "A" : 'A2_tone.wav',
        "A#": 'A#2_tone.wav',
        "B" : 'B2_tone.wav',
    },
    "3" : {
        "C" : 'C3_tone.wav',
        "C#": 'C#3_tone.wav',
        "D" : 'D3_tone.wav',
        "D#": 'D#3_tone.wav',
        "E" : 'E3_tone.wav',
        "F" : 'F3_tone.wav',
        "F#": 'F#3_tone.wav',
        "G" : 'G3_tone.wav',
        "G#": 'G#3_tone.wav',
        "A" : 'A3_tone.wav',
        "A#": 'A#3_tone.wav',
        "B" : 'B3_tone.wav',
    },
    "4" : {
        "C" : 'C4_tone.wav',
        "C#": 'C#4_tone.wav',
        "D" : 'D4_tone.wav',
        "D#": 'D#4_tone.wav',
        "E" : 'E4_tone.wav',
        "F" : 'F4_tone.wav',
        "F#": 'F#4_tone.wav',
        "G" : 'G4_tone.wav',
        "G#": 'G#4_tone.wav',
        "A" : 'A4_tone.wav',
        "A#": 'A#4_tone.wav',
        "B" : 'B4_tone.wav',
    },
    "5" : {
        "C" : 'C5_tone.wav',
    },
}

In [16]:
code_spans = {
    "maj":    [0, 4, 3],   # 例: C → E (4半音), E → G (3半音)
    "maj6":   [0, 4, 3, 2],  # 例: C E G A
    "dom7":   [0, 4, 3, 3],  # ドミナントセブンス（例: G7 = G B D F）
    "majadd9":[0, 4, 3, 7], # majに長9度を足したもの。
    "maj6-9": [0, 4, 3, 2, 5], # 長9度は長2度（全音一つ)の音階をオクターブ上か下に動かしたもの[-10, 0, 4, 3, 2]も同じ名前になっている
    "maj7":   [0, 4, 3, 4],  # 例: C E G B
    "maj7f5": [0, 4, 2, 4],  # 例: C E F# B 7th フラット5
    "maj9":   [0, 4, 3, 4, 3], # maj7に長9度を足したもの。
    "aug":  [0, 4, 4],   # オーグメント：拡張された、夢幻的な響き
    "min" : [0, 3, 4],   # 例: A → C (3), C → E (4)
    "dim" : [0, 3, 3],   # ディミニッシュ：不安定、緊張感
    "min7": [0, 3, 4, 3],  # 例: A C E G
    "dim7": [0, 3, 3, 3],  # ディミニッシュセブンス（完全に均等な構成）
    "m7b5": [0, 3, 3, 4],  # ハーフディミニッシュ（マイナー7♭5）
    "sus2": [0, 2, 5],     # サス2：ルート → 2度 → 完全5度
    "sus4": [0, 5, 2],     # サス4：ルート → 4度 → 完全5度
}

In [17]:
notes_and_octaves = [
    ("C", 2), ("C#", 2), ("D", 2), ("D#", 2), ("E", 2), ("F", 2), ("F#", 2), ("G", 2), ("G#", 2), ("A", 2), ("A#", 2), ("B", 2),
    ("C", 3), ("C#", 3), ("D", 3), ("D#", 3), ("E", 3), ("F", 3), ("F#", 3), ("G", 3), ("G#", 3), ("A", 3), ("A#", 3), ("B", 3),
    ("C", 4), ("C#", 4), ("D", 4), ("D#", 4), ("E", 4), ("F", 4), ("F#", 4), ("G", 4), ("G#", 4), ("A", 4), ("A#", 4), ("B", 4),
    ("C", 5)
]

### 主要なコードの構成音間の半音数

In [18]:
## コード音声生成。
# 関数：コード構成音を得る
def get_chord_notes(root, code_type):
    root_index = NOTES.index(root)
    intervals = code_spans[code_type]
    notes = []
    idx = root_index
    for step in intervals:
        notes.append(NOTES[idx % 12])
        idx += step
    return notes



In [23]:
import os
from pydub import AudioSegment
import os

# 出力用フォルダ
output_dir = "output_chords"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# 波形とサンプルレートの読み込み
waveforms = []
samplerate = None

for filepath in wav_files:
    data, sr = sf.read(filepath)
    if samplerate is None:
        samplerate = sr
    elif sr != samplerate:
        raise ValueError(f"Sample rate mismatch: {filepath}")
    
    # モノラル化（ステレオの場合）
    if data.ndim > 1:
        data = data.mean(axis=1)
    
    waveforms.append(data)

# 長さを揃える（短い波形はゼロパディング）
max_len = max([len(w) for w in waveforms])
padded = [np.pad(w, (0, max_len - len(w))) for w in waveforms]

# NumPy配列にして合計
stacked = np.vstack(padded)
mixed = np.sum(stacked, axis=0)

# 正規化（±1 の範囲に収める）
max_val = np.max(np.abs(mixed))
if max_val > 1:
    mixed = mixed / max_val

# 保存
sf.write("mixed_output.wav", mixed, samplerate)
print("✔️ 書き出しました: mixed_output.wav")

In [27]:
# 音名リスト
NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
# コード生成（オクターブ "2" を使う前提）
octave = "2"
for root in NOTES:
    for code_type in code_spans:
        chord_notes = get_chord_notes(root, code_type)

        try:
            # 音声読み込み
            segments = [AudioSegment.from_wav("sounds/" + soundfiles[octave][note]) for note in chord_notes]

            # 長さを揃えてミックス
            max_len = max(len(seg) for seg in segments)
            segments = [seg + AudioSegment.silent(duration=max_len - len(seg)) for seg in segments]

            # 重ねる
            chord_audio = segments[0]
            for seg in segments[1:]:
                chord_audio = chord_audio.overlay(seg)

            # 書き出し
            filename = f"code_{root.replace('#', 's')}{code_type}.wav"  # ファイル名に "#" は使わない方が良い
            chord_audio.export(os.path.join(output_dir, filename), format="wav")
            print(f"✔️ Exported: {filename}")
        except KeyError as e:
            print(f"⚠️ Skipping {root} {code_type}: Missing sound file for {e}")

✔️ Exported: code_Cmaj.wav
✔️ Exported: code_Cmaj6.wav
✔️ Exported: code_Cdom7.wav
✔️ Exported: code_Cmajadd9.wav
✔️ Exported: code_Cmaj6-9.wav
✔️ Exported: code_Cmaj7.wav
✔️ Exported: code_Cmaj7f5.wav
✔️ Exported: code_Cmaj9.wav
✔️ Exported: code_Caug.wav
✔️ Exported: code_Cmin.wav
✔️ Exported: code_Cdim.wav
✔️ Exported: code_Cmin7.wav
✔️ Exported: code_Cdim7.wav
✔️ Exported: code_Cm7b5.wav
✔️ Exported: code_Csus2.wav
✔️ Exported: code_Csus4.wav
✔️ Exported: code_Csmaj.wav
✔️ Exported: code_Csmaj6.wav
✔️ Exported: code_Csdom7.wav
✔️ Exported: code_Csmajadd9.wav
✔️ Exported: code_Csmaj6-9.wav
✔️ Exported: code_Csmaj7.wav
✔️ Exported: code_Csmaj7f5.wav
✔️ Exported: code_Csmaj9.wav
✔️ Exported: code_Csaug.wav
✔️ Exported: code_Csmin.wav
✔️ Exported: code_Csdim.wav
✔️ Exported: code_Csmin7.wav
✔️ Exported: code_Csdim7.wav
✔️ Exported: code_Csm7b5.wav
✔️ Exported: code_Cssus2.wav
✔️ Exported: code_Cssus4.wav
✔️ Exported: code_Dmaj.wav
✔️ Exported: code_Dmaj6.wav
✔️ Exported: code_Ddom7.wav